The file format is as follows. In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file. Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "-" sign denoting logical "not". For example, the second line of the first data file is "-16808 75250", which indicates the clause ¬x16808∨x75250.

Your task is to determine which of the 6 instances are satisfiable, and which are unsatisfiable. In the box below, enter a 6-bit string, where the ith bit should be 1 if the ith instance is satisfiable, and 0 otherwise. For example, if you think that the first 3 instances are satisfiable and the last 3 are not, then you should enter the string 111000 in the box below.

In [ ]:
# Solution 1

import random

text = open("/Users/yuqinglu/E_Learning/Coursera_Algorithms/datasets/2sat1.txt")
line = text.readline()
varnum = int(line)
line = text.readline()
clauses = []
while line:
    a = line.split()
    clauses.append((int(a[0]), int(a[1])))
    line = text.readline()

print len(clauses)

def sign(n):
    return n>0

def solve_2SAT(varnum, clauses):
    for sth in range(0,4):
        x = [-1]*(varnum/2) +[1]*(varnum/2)
        random.shuffle(x)
        print "finish initialization for # of pass:", sth
        for sth2 in range(1, 3*varnum):
            if sth2 % 10000 == 0:
                print sth2
            sat = True
            n = random.randint(0, varnum-1)
            for i in range(0, varnum):
                clause = clauses[(i+n) % varnum]
                if (sign(x[abs(clause[0])-1]) != sign(clause[0])) and (sign(x[abs(clause[1])-1]) != sign(clause[1])):
                    sat = False
                    break
            if sat:
                print "satisfied!"
                return
            k = random.randint(0,1)
            x[abs(clause[k])-1] = -x[abs(clause[k])-1]
    print "All complete, not satified"
    return None
            
solve_2SAT(varnum, clauses)

In [ ]:
# Solution 2 super fast

import math
import random
import gc
import sys
import threading
from StringIO import StringIO

def make_sat_graph(clauses):
    n = len(clauses)
    def var_index(var):
        if var < 0: return n - var
        else: return var
    res = ''
    for clause in clauses:
        res += '%i %i\n' % (var_index(-clause[0]), var_index(clause[1]))
        res += '%i %i\n' % (var_index(-clause[1]), var_index(clause[0]))
    return res
        
        
################################################################################
#######      Kosaraju's SSC algorithm implementation from part 1          ######
################################################################################

def readDirectedGraph(str):
    f = StringIO(str)
    
    adjlist = []
    adjlist_reversed = []
    
    line = f.readline()
    while line != '':
        num1, num2 = line.split()
        v_from = int(num1)
        v_to = int(num2)
        max_v = max(v_from, v_to)
        
        while len(adjlist) < max_v:
            adjlist.append([])
        while len(adjlist_reversed) < max_v:
            adjlist_reversed.append([])
            
        adjlist[v_from-1].append(v_to-1)
        adjlist_reversed[v_to-1].append(v_from-1)
        
        line = f.readline()
            
    return adjlist, adjlist_reversed


t = 0
s = None
n = 0
explored = None
leader = None
current_ssc = None
contradictory_ssc = None
sorted_by_finish_time = None

def DFS_Loop_1(graph_rev, n):
    
    global t, explored, sorted_by_finish_time
    
    t = 0
    explored = [False]*n
    sorted_by_finish_time = [None]*n
    
    for i in reversed(range(n)):
        if not explored[i]:
            DFS_1(graph_rev, i)
                        
            
def DFS_1(graph_rev, i):
    
    global t, explored
    
    explored[i] = True
    
    for v in graph_rev[i]:
        if not explored[v]:
            DFS_1(graph_rev, v)
    
    sorted_by_finish_time[t] = i
    t += 1
    
    
def DFS_Loop_2(graph):
    
    global current_ssc, explored, contradictory_ssc, sorted_by_finish_time
    
    explored = [False]*len(graph)
    
    for i in reversed(range(len(graph))):
        if not explored[sorted_by_finish_time[i]]:
            scc_size = 0
            # Here we collect all the members
            # of the next SCC using DFS.
            current_ssc = set()
            contradictory_ssc = False
            DFS_2(graph, sorted_by_finish_time[i])
            if contradictory_ssc: break
            
    return contradictory_ssc
    
    
def DFS_2(graph, i):
    
    global explored, current_ssc, contradictory_ssc
    
    explored[i] = True
    current_ssc.add(i)
    
    # Check for unsatisfabilty indicator
    if i < n:
        if (i+n) in current_ssc:
            contradictory_ssc = True
    elif (i-n) in current_ssc:
        contradictory_ssc = True
    
    for v in graph[i]:
        if not explored[v]:
            DFS_2(graph, v)
    

def kosaraju_contradictory_ssc(graph, graph_rev):
    
    DFS_Loop_1(graph_rev, len(graph))
    contradictory_ssc = DFS_Loop_2(graph)
    
    return contradictory_ssc


def main():

    global n

    for i in xrange(1, 7):
        print 'file %i' % i
        f = open('2sat%i.txt' % i)
        n = int(f.readline())
        clauses = [[int(x) for x in line.split()] for line in f]
        
        sat_graph = make_sat_graph(clauses)
        graph, graph_rev = readDirectedGraph(sat_graph)
        contradictory_ssc = kosaraju_contradictory_ssc(graph, graph_rev)
        res = 'unsatisfiable' if contradictory_ssc else 'satisfiable'
        
        print 'result: %s\n' % res
        
        gc.collect()


if __name__ == '__main__':
    threading.stack_size(67108864) # 64MB stack
    sys.setrecursionlimit(2 ** 20)  # approx 1 million recursions
    thread = threading.Thread(target = main) # instantiate thread object
    thread.start() # run program at target

In [ ]:
# My python3 conversion of above script

import math
import random
import gc
import sys
import threading
from io import StringIO


def make_sat_graph(clauses):
    n = len(clauses)

    def var_index(var):
        return var if var >= 0 else n - var
    
    res = ''
    for clause in clauses:
        res += '{} {}\n'.format(var_index(-clause[0]), var_index(clause[1]))
        res += '{} {}\n'.format(var_index(-clause[1]), var_index(clause[0]))
    return res


def read_directed_graph(str_):
    f = StringIO(str_)
    
    adjlist = []
    adjlist_reversed = []
    
    line = f.readline()
    while line != '':
        num1, num2 = line.split()
        v_from = int(num1)
        v_to = int(num2)
        max_v = max(v_from, v_to)
        
        while len(adjlist) < max_v:
            adjlist.append([])
        while len(adjlist_reversed) < max_v:
            adjlist_reversed.append([])
            
        adjlist[v_from-1].append(v_to-1)
        adjlist_reversed[v_to-1].append(v_from-1)
        line = f.readline()
            
    return adjlist, adjlist_reversed


t = 0
s = None
n = 0
explored = None
leader = None
current_ssc = None
contradictory_ssc = None
sorted_by_finish_time = None
    

def dfs_loop_1(graph_rev, n):
    global t, explored, sorted_by_finish_time
    
    t = 0
    explored = [False]*n
    sorted_by_finish_time = [None]*n
    
    for i in reversed(range(n)):
        if not explored[i]:
            dfs_1(graph_rev, i)
    
    
def dfs_1(graph_rev, i):
    global t, explored
    
    explored[i] = True
    for v in graph_rev[i]:
        if not explored[v]:
            dfs_1(graph_rev, v)
    
    sorted_by_finish_time[t] = i
    t += 1
    

def dfs_loop_2(graph):
    
    global current_ssc, explored, contradictory_ssc, sorted_by_finish_time
    
    explored = [False]*len(graph)
    
    for i in reversed(range(len(graph))):
        if not explored[sorted_by_finish_time[i]]:
            scc_size = 0
            # Here we collect all the members
            # of the next SCC using DFS.
            current_ssc = set()
            contradictory_ssc = False
            dfs_2(graph, sorted_by_finish_time[i])
            if contradictory_ssc: break
            
    return contradictory_ssc
    
    
def dfs_2(graph, i):
    
    global explored, current_ssc, contradictory_ssc
    
    explored[i] = True
    current_ssc.add(i)
    
    # Check for unsatisfabilty indicator
    if i < n:
        if (i+n) in current_ssc:
            contradictory_ssc = True
    elif (i-n) in current_ssc:
        contradictory_ssc = True
    
    for v in graph[i]:
        if not explored[v]:
            dfs_2(graph, v)
    

def kosaraju_contradictory_ssc(graph, graph_rev):
    
    dfs_loop_1(graph_rev, len(graph))
    contradictory_ssc = dfs_loop_2(graph)
    
    return contradictory_ssc


def main():
    global n
    
    for i in range(1, 7):
        print('file {}'.format(i))
        with open('datasets/2sat{}.txt'.format(i)) as f:
            n = int(f.readline())
            clauses = [[int(x) for x in line.split()] for line in f]
        sat_graph = make_sat_graph(clauses) 
        graph, graph_rev = read_directed_graph(sat_graph)
        contradictory_ssc = kosaraju_contradictory_ssc(graph, graph_rev)
        res = 'unsatisfiable' if contradictory_ssc else 'satisfiable'
        
        print('result: {}\n'.format(res))
        
        gc.collect()


if __name__ == '__main__':
    threading.stack_size(67108864) # 64MB stack
    sys.setrecursionlimit(2 ** 20)  # approx 1 million recursions
    thread = threading.Thread(target = main) # instantiate thread object
    thread.start() # run program at target